<h1> Introducing Pymgrid, the SmartGrid Management Simulator ! </h1>

Pymgrid is an open-source python package developped by Total R&D to simulate Smartgrids (or Microgrids) and track their profitability over a year. The package is specifically designed to use of Reinforcement Learning, or Rule Based methods or Optimization approaches to compare them in terms of profitability. 

<h3> How to use Pymgrid in general </h3> 

While the Pymgrid package can generate many kinds of microgrids, <b> the 3 buildings you will be working with are specific to this Hi!ckathon, you wont be able to find them in the GitHub. </b>

Load the 3 buildings with the following line of code :

In [1]:
import pickle

with open('building_1.pkl', 'rb') as f:
    building_1 = pickle.load(f)

with open('building_2.pkl', 'rb') as f:
    building_2 = pickle.load(f)
    
with open('building_3.pkl', 'rb') as f:
    building_3 = pickle.load(f)

buildings = [building_1, building_2, building_3]

By default, the buildings are loaded in their "training" version, a later line of code will show you how to switch to the "testing" version

<b> Keep in mind that the buildings are MicroGrid objects, any function that works for a MicroGrid object will work on the 3 buildings. </b>

You can look at some additional information about your buildings with the line of code below

In [2]:
for building in buildings:
    print(building.architecture)

{'PV': 1, 'battery': 1, 'genset': 0, 'grid': 1}
{'PV': 1, 'battery': 1, 'genset': 0, 'grid': 1}
{'PV': 1, 'battery': 1, 'genset': 1, 'grid': 1}


The 2 first buildings are both equiped with a battery, a PV and the ability to import power from the public electricity grid at any moment. The third building is more challenging since it only has a "Weak Grid", meaning it is sometimes cut-off from the public grid and thus unable to import electricity from it. This third building is also equipped with a fuel-based generator (Genset), and your algorithm will have to learn to deal with both the local tools and the availability of the public grid.

Another mandatory function is the reset() function. It allows you to reset a building back to the beginning of the year, as if no decisions were made by your algorithm.

In [3]:
buildings[0].reset()
buildings[1].reset()
buildings[2].reset()

The reset function also allows you to switch your building into the "test" mode to evaluate Profitability. This is how to do it :

In [4]:
buildings[0].reset(testing=True)
buildings[1].reset(testing=True)
buildings[2].reset(testing=True)

<font color='red'> <h3> Hi!ckathon Rules for all approaches </h3> </font>

<font color='red'> <b> It is forbidden to :
* Train your RL or Deep RL algorithm on the test version of the buildings 
* Make any modifications to the DiscreteEnvironment script and/or building pickle files
* Submit an algorithm you've simply copy/pasted from the Github without any modification 
    </b> </font>

<h3> How to use Pymgrid for the Expertise Path 

Pymgrid is designed to allow you to easily implement rules based on business knowledge to manage your energy consumption ! 

<h4> Interacting with the buildings in Pymgrid </h4>

At each time step (each hour) a decision has to be made to supply a building's demand in energy (called load). To do so, you must fill for each time step a <b> control dictionary </b>.

The control dictionnary is sturcutred in the following way :
* Each key is one of the possible actions you can take regarding the building's equippement or the public grid (with 2 additional exception keys that are not actions )
* Each value is the amount of energy in kW that the action is leveraging

The following line of code prints all possible actions we have at our disposal for each building :

In [5]:
buildings[0].get_control_dict()

['load',
 'pv_consummed',
 'pv_curtailed',
 'pv',
 'battery_charge',
 'battery_discharge',
 'grid_import',
 'grid_export']


* <b> PV Consumed: </b> How much energy was used from the PV to supply the load
* <b> PV Curtailed: </b> How much energy did we have to deliberitaly not produce to supply the load (since solar power cannot be stored, look up curtailement to know more)
* <b> Battery Charge: </b> Self-explanatory
* <b> Battery Discharge : </b> Self-explanatory
* <b> Grid Import : </b> Grid here refers to the public electricity grid 
* <b> Grid Export : </b> In case of excessive energy generation (or overgeneration) you can export the excess to the public grid
* <b> Load (Not an Action) : </b> The demand in energy of the building at the current time step
* <b> PV (Not an Action) : </b> The overall PV for the current time step



Below is an example of a control dictionary you can run at given time step : 

In [6]:
 control_dict = {
     'battery_charge': 0,
     'battery_discharge': 150,
     'grid_import': 200,
     'grid_export': 0,
     'pv_consummed': 53,
     }

<b> Notice that for a single time step, the energy management decision you can take can be a combination of many actions at once ! </b>

The following line of code allows you to know the current state of the building

In [7]:
buildings[0].get_updated_values()

{'load': 2.3,
 'hour': 0,
 'pv': 0.0,
 'battery_soc': 0.2,
 'capa_to_charge': 13.3,
 'capa_to_discharge': 0.0,
 'grid_status': 1.0,
 'grid_co2': 0.180924313,
 'grid_price_import': 0.22,
 'grid_price_export': 0.0}

Let's describe the content :
* We are at hour=0, meaning this is the initial step, no decision has been made 
* Our first demand in energy is load=2.3 kW
* Our PV has no solar energy to offer
* Our battery has a State of Charge of 20% (meaning it is at 20% of full charge)
* Our battery has the capacity to charge 13.3 kW
* The public grid is currently available grid_status=1 (This will always be the case for the 2 first buildings)
* The 3 last values are respectively :
    * kCO2 generated per kW by importing from the grid --> The more CO2 is
    * Electricity price per kWh when importing from the grid
    * Electricity price per kWh when exporting from the grid
    * All 3 can vary over time, and are used to calculate the cost of a decision you make
    
    

The following line of code makes an energy management decision by running your control dictionary, it will modify the state of the building and will return the same information as in the previous line of code.

In [8]:
building_new_data = buildings[0].run(control_dict)
print(building_new_data)

{'load': 2.2749179536232047, 'hour': 0, 'pv': 0.0, 'battery_soc': 0.2, 'capa_to_charge': 13.333333333333332, 'capa_to_discharge': 0.0, 'grid_status': 1.0, 'grid_co2': 0.18330306, 'grid_price_import': 0.22, 'grid_price_export': 0.0}


Of course, any decision you take will generate a cost. The following line of code allows you to know the cost of the most recent decision made for the building :

In [9]:
buildings[0].get_cost()

10.729409916895865

Don't forget to reset the building if you need to restart from the beginning of the year ! 

In [10]:
buildings[0].reset()

<h4> What your expertise Rule Based approach should be like </h4>

Your goal will be implement rules, or a series of if-else conditions that take into account the state of your building and decide which action should be made in priority. Let's take a look at the building state variables to get a better idea

In [11]:
buildings[0].get_updated_values()

{'load': 2.3,
 'hour': 0,
 'pv': 0.0,
 'battery_soc': 0.2,
 'capa_to_charge': 13.3,
 'capa_to_discharge': 0.0,
 'grid_status': 1.0,
 'grid_co2': 0.180924313,
 'grid_price_import': 0.22,
 'grid_price_export': 0.0}

You will notice that the price of importing electricity varies over time, this implies that it could be interesting to have a rule that monitors variations in electricity price to decide whether we should import more or import less, and in case we import less we would need to have a full battery to use to supply the load. 

<b> A naive example to get your started can be found below </b>

In [ ]:
def naive_rule_based_strategy(building):
    
    building_data = building.get_updated_values()

    total_building_cost = 0

    while building.done == False:
        load = building_data['load']
        pv = building_data['pv']
        capa_to_charge = building_data['capa_to_charge']
        capa_to_dischare = building_data['capa_to_discharge']

        p_disc = max(0, min(load-pv, capa_to_dischare, building.battery.p_discharge_max))
        p_char = max(0, min(pv-load, capa_to_charge, building.battery.p_charge_max))

        if load - pv >=  0:

            control_dict = {'battery_charge': 0,
                                'battery_discharge': p_disc,
                                'grid_import': max(0, load-pv-p_disc),
                                'grid_export':0,
                                'pv_consummed': min(pv, load),
                           }


        if load - pv <  0:

            control_dict = {'battery_charge': p_char,
                                'battery_discharge': 0,
                                'grid_import': 0,
                                'grid_export': max(0,pv-load-p_char),#abs(min(load-pv,0)),
                                'pv_consummed': min(pv, load+p_char),
                           }

        building_data = building.run(control_dict)
        total_building_cost += building.get_cost()
    
    return total_building_cost

<font color='red'> <h4> Compulsory Rules for your submission </h4> </font>

* It is forbidden to submit the exact results of the RBC Benchmark implemented in Pymgrid or copy the exact same approach, although the code can serve as an inspiration to implement your solution

<font color='red'> The details for evaluation for this approach are laid out in the Evaluation Notebook Template </font>

<h3> How to use Pymgrid for the Reinforcement Learning Path </h3>

If you are familiar with Reinforcement Learning in Python, then you know that OpenAI Gym is the standard framework for most implementations. Pymgrid is no different, and it provides you with the abilitty to "Gymify" your buildings.

In [12]:
### Import the Gym environnement with finite States & Actions
import DiscreteEnvironment as DiscreteEnvironment 

In [13]:
### Gymify your building MicroGrid object
env_config={'building':buildings[0]}
building_environment = DiscreteEnvironment.Environment(env_config)

Now  the building_environment variable behaves exactly like an OpenAI Gym environment. For example we can take an action:

In [19]:
building_environment.step(0)

((-6, 0.4), -9.400159089330588, False, {})

It returns the next state, the reward, if we reached the end of the episode, and additionnal info. Let's take a look at the actions and states.

In [24]:
building_environment.action_space

Discrete(5)

There are 5 different actions available :\
-0: Charge the battery from the PV\
-1: Discharge the battery\
-2: Import from the grid\
-3: Export to the grid\
-4: Charge the battery from the grid\

In [25]:
building_environment.observation_space

MultiDiscrete([ 45 100])

The environment returns two information : \
-The current net load\
-The current state of charge (out of 100) of the battery

Once you're done with training, you can switch to the test environment here.

In [17]:
building_environment.reset(testing=True)

(-9, 0.2)

<b> Be careful, the rewards returned by the Gym environment are negative ! Don't forget to multiply by -1 the total reward as the Profitability you will need to submit needs to be positive ! </b>

<b> To get started you can check out a Q-Learning implementation on Pymgrid in Github : https://github.com/Total-RD/pymgrid/blob/master/notebooks/A%20Q-Learning%20Example%20with%20PymGrid.ipynb </b>

<font color='red'> <h4> Compulsory Rules for your submission </h4> </font>

* It is compulsory that you work with DiscreteEnvironment if you want to implement RL approaches with no Deep Learning
* It is forbidden to use Deep Learning approaches with the DiscreteEnvironment
* If you want to use Q-Learning based approaches, you will have to use the Q-Table defined below

In [15]:
##### Represents an input Gym environment Building as a Q-Table

import numpy 

def init_qtable(env, nb_action):
    
    state = []
    Q = {}

    for i in range(-int(env.mg.parameters['PV_rated_power']-1),int(env.mg.parameters['load']+2)):
        
        for j in np.arange(round(env.mg.battery.soc_min,1),round(env.mg.battery.soc_max+0.1,1),0.1):
            
            j = round(j,1)
            state.append((i,j)) 

    #Initialize Q(s,a) at zero
    for s in state:

        Q[s] = {}

        for a in range(nb_action):

            Q[s][a] = 0

    return Q

<font color='red'> The details for evaluation for this approach are laid out in the Evaluation Notebook Template </font>

<h3> How to use Pymgrid for the Deep Reinforcement Learning Path </h3>

If you are familiar with Reinforcement Learning in Python, then you know that OpenAI Gym is the standard framework for most implementations. Pymgrid is no different, and it provides you with the abilitty to "Gymify" your buildings.

In [2]:
### Import the Gym environnement with continuous States & discrete actions
from pymgrid.Environments.pymgrid_cspla import MicroGridEnv

In [5]:
building_environment = MicroGridEnv(env_config={'microgrid':buildings[0],"testing":False})

Now  the building_environment variable behaves exactly like an OpenAI Gym environment. For example we can take an action:

In [6]:
building_environment.step(0)

(array([ 2.27491795,  0.        ,  0.        ,  0.2       , 13.33333333,
         0.        ,  1.        ,  0.18330306,  0.22      ,  0.        ]),
 -2.275224306041345,
 False,
 {})

It returns the next state (a vector in this case), the reward, a boolean indicating if we reached the end of the episode and potential additionnal info.

Let's take a look at the possible actions you can take in this environment:

In [24]:
building_environment.action_space

Discrete(5)

There are 5 different actions available :\
-0: Charge the battery from the PV\
-1: Discharge the battery\
-2: Import from the grid\
-3: Export to the grid\
-4: Charge the battery from the grid\

About the state vectors, they represent information about the building's equiment and information about the public grid. You can look at this information with the following function :

In [7]:
## Notice that this function is used on the MicroGrid object not the OpenAI Gym object
buildings[0].get_updated_values()

{'load': 2.2749179536232047,
 'hour': 0,
 'pv': 0.0,
 'battery_soc': 0.2,
 'capa_to_charge': 13.333333333333332,
 'capa_to_discharge': 0.0,
 'grid_status': 1.0,
 'grid_co2': 0.18330306,
 'grid_price_import': 0.22,
 'grid_price_export': 0.0}

Let's describe the content :
* We are at hour=0, meaning this is the initial step, no decision has been made 
* Our first demand in energy is load=2.3 kW
* Our PV has no solar energy to offer
* Our battery has a State of Charge of 20% (meaning it is at 20% of full charge)
* Our battery has the capacity to charge 13.3 kW
* The public grid is currently available grid_status=1 (This will always be the case for the 2 first buildings)
* The 3 last values are respectively :
    * kCO2 generated per kW by importing from the grid --> The more CO2 is
    * Electricity price per kWh when importing from the grid
    * Electricity price per kWh when exporting from the grid
    * All 3 can vary over time, and are used to calculate the cost of a decision you make
    
    

<b> All this information at a given time step defines the state vector of the environment </b>

<b> How does my favourite Deep Learning RL library interact with Pymgrid ? </b>

The MicroGridEnv object acts as a Custom Gym Environment, this means that when you create your agent using the Deep RL library of your choice, you will specify MicroGridEnv as an environment and will provide a config dictionnary to specificy to the agent exactly which of the 3 buildings he will be working with.


Each Deep RL library has it's own way of initializing an agent with a CustomEnvironment, so choose the one you are most confortable with ! Just make sure it's gym conpatible.

After you've initialized your agent as described, you will be able to work as you usally do with Gym. <b> Don't forget to switch on the test buildings ! </b>

<font color='red'> <h4> Compulsory Rules for your submission </h4> </font>

* It is compulsory that you work exclusively with the MicroGridEnv imported from pymgrid.Environments.pymgrid_cspla. Any other environment used will be rejected

<font color='red'> The details for evaluation for this approach are laid out in the Evaluation Notebook Template </font>

<h3> Note on the Optimizer's Path </h3>

As there is a very vast and diverse number of ways one can formulate an Optimization under constraints problem, it is difficult to guide you along a specific methodology to use Pymgrid. 

So we deliberitaly leave all possibilities open, evaluation rules will be specific to your approach.

If you wish to solve the problem this way, coaches will be not able to guarantee that your solution is really on the right path or not. You'll have to really know what you are doing.